# Imports

In [1]:
import requests
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import re
from datetime import datetime
import pandas as pd
from connection_keys import connection, alchemy_connection

### Setup para abrir o navegador com o Selenium e puxar a primeira página

In [72]:
#--| Setup
options = Options()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
options.add_argument('--disable-extensions')
#options.add_argument("--headless")
browser = webdriver.Chrome(executable_path=r'chromedriver.exe', options=options)
#--| Parse or automation
browser.get('https://www.kabum.com.br/hardware/placa-de-video-vga?pagina=1')
html = soup(browser.page_source, 'html.parser')
# browser.implicitly_wait(5)

### Configuiração para percorer as páginas e salvar todos dados com a taga referente ao produto em uma lista

In [73]:
pagina = 0
html_produtos = []
flag = 0

while flag == 0:
    items = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})
    if items != []:
        try:
            pagina = pagina + 1
            browser.get(f'https://www.kabum.com.br/hardware/placa-de-video-vga?pagina={pagina}')
            html = soup(browser.page_source, 'html.parser')
        except Exception: break
        html_produtos.append(items)
    else:
        break

### Tranformação da lista de listas em uma lista normal para percorer com mais facildiade os atributos do produto

In [74]:
flat_list = []
for sublist in html_produtos:
    for item in sublist:
        flat_list.append(item)

### Percorrendo a lista, capurando e salvando cada atributo no dicionario

In [75]:
now = datetime.now()
produtos = []

for container in flat_list:
    
    # Estreutura de para salvar
    marca = container.div.img['alt']
    nome_produto = container.find('a', {"class":"sc-fzoLsD gnrNhT item-nome"}).text

    if container.find('div', {"class":"sc-fzpjYC hKlXgM"}) != None:
        preco_antigo_cartao = (container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2]
    else: 
        preco_antigo_cartao = '0'
    preco_atual_cartao =  (container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1]
    preco_boleto = (container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1]
    
    image = container.find('div', {"class":"sc-fznKkj fEpBwY"})
    if "indisponivel" in image.img['src']:
        disponibilidade = 'Indisponivel'
    elif "disponivel" in image.img['src']:
        disponibilidade = 'Disponivel'

    produto = {
        "marca" : marca,
        "nome_produto": nome_produto,
        "preco_antigo_cartao": preco_antigo_cartao,
        "preco_atual_cartao": preco_atual_cartao,
        "preco_boleto": preco_boleto,
        "disponibilidade": disponibilidade,
        "data" : now.strftime("%Y-%m-%d"),
        "hora":  now.strftime("%H:%M:%S"),
        "loja": "Kabum" 
    }
    produtos.append(produto)


### Transformando o dict em Dataframe e visualizando uma amostra dos dados

In [82]:
df = pd.DataFrame(produtos)
df.sample(10)

,marca,nome_produto,preco_antigo_cartao,preco_atual_cartao,preco_boleto,disponibilidade,data,hora,loja
315,MSI,Placa de Vídeo MSI AMD Radeon RX 580 Armor MK2...,0,"1.011,65","859,90",Indisponivel,2020-09-10,19:12:39,Kabum
97,Galax,Placa de Vídeo Galax NVIDIA GeForce RTX 2080 S...,"8.235,18","7.411,65","6.299,90",Disponivel,2020-09-10,19:12:39,Kabum
96,Evga,Placa de Vídeo EVGA NVIDIA GeForce GT 1030 SC ...,"941,06","846,94","719,90",Disponivel,2020-09-10,19:12:39,Kabum
132,Galax,Placa de Vídeo VGA Galax NVIDIA GeForce GT 103...,"894,00","823,41","699,90",Disponivel,2020-09-10,19:12:39,Kabum
57,Gigabyte,Placa de Vídeo Gigabyte NVIDIA GeForce RTX 208...,0,"7.646,94","6.499,90",Disponivel,2020-09-10,19:12:39,Kabum
373,Evga,Placa de Vídeo EVGA NVIDIA GeForce GTX 1660 Bl...,"2.352,82","2.117,53","1.799,90",Indisponivel,2020-09-10,19:12:39,Kabum
233,Gainward,Placa de Vídeo Gainward NVIDIA GeForce RTX 208...,0,"12.623,41","10.729,90",Disponivel,2020-09-10,19:12:39,Kabum
306,Gigabyte,Placa de Vídeo Gigabyte NVIDIA GeForce GT 1030...,"830,90","823,41","699,90",Indisponivel,2020-09-10,19:12:39,Kabum
65,Evga,Placa de Video EVGA GeForce RTX 2070 Super XC ...,"4.771,65","4.646,94","3.949,90",Disponivel,2020-09-10,19:12:39,Kabum
286,Evga,Placa de Vídeo EVGA NVIDIA GeForce GTX 1050 Ti...,0,"823,41","699,90",Indisponivel,2020-09-10,19:12:39,Kabum



### Criando a conecão com o banco e salvando os dados

In [ ]:
engine = alchemy_connection('db_hardware')
df.to_sql('graphic_card', con=engine, if_exists='append', index = False)

----------------------------------------------------------------------------------------------------

In [13]:
# containers = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})

In [80]:
# images = html.findAll('div', {"class":"sc-fznKkj fEpBwY"})

# # images.img['src']
# for image in images:
#     print(image.img['src'])
#     # if "indisponivel" in image.img['src']:
#     #     print('Indisponivel')
#     # elif "disponivel" in image.img['src']:
#     #     print('Disponivel')

In [4]:
# preco_antigo_cartao = float((container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2].replace('.', '').replace(',', '.'))
# preco_atual_cartao =  float((container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1].replace('.', '').replace(',', '.'))
# preco_boleto = float((container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1].replace('.', '').replace(',', '.'))


In [5]:
# print(preco_antigo_cartao)
# # print(re.search('^(([1-9]\d{0,2}(\.\d{3})*)|(([1-9]\.\d*)?\d))(\,\d\d)?', preco_atual_cartao).group())
# print(preco_atual_cartao)
# print(preco_boleto)